In [32]:
# load dataset 
import pandas as pd 

path = './song-lyrics/lyrics-data.csv'

dataframe = pd.read_csv(path)

In [33]:
# filter Idiom for olny english lyrics
dataframe = dataframe[dataframe['Idiom'] == 'ENGLISH']

In [36]:
# normalize lycris to lowercase  
dataframe['Lyric'] = dataframe['Lyric'].apply(lambda lyric: lyric.lower()) 